# Import

In [1]:
import sys

root_dir = '../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import nn, optim
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

from modules import losses, models, samplers, regularizers, evaluators, trainers, datasets, distributions

# DataSet

In [2]:
dataset = datasets.ML100k()
n_user = dataset.n_user
n_item = dataset.n_item
train_set, test_set = dataset.get_train_and_test_set()

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.LongTensor(test_set).to(device)

# Evaluator

In [3]:
ks = [5, 10, 50]
recall = evaluators.RecallEvaluator(test_set, ks)

# Sampler

In [4]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

# Model

In [5]:
# Hyperparameters
lr = 1e-3
n_dim = 10
n_batch = 256
n_epoch = 50
valid_per_epoch = 10
n_item_sample = 30
n_user_sample = 30
no_progressbar = True

# distributiuons
# dist1 = distributions.Empirical()
dist2 = distributions.Gaussian()
dist3 = distributions.Gamma()

# models
model0 = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
# model1 = models.MutualProximityCML(n_user, n_item, dist1, n_dim, n_item_sample, n_user_sample).to(device)
model2 = models.MutualProximityCML(n_user, n_item, dist2, n_dim, n_item_sample, n_user_sample).to(device)
model3 = models.MutualProximityCML(n_user, n_item, dist3, n_dim, n_item_sample, n_user_sample).to(device)

# learning late optimizer
optimizer0 = optim.Adam(model0.parameters(), lr=lr)
optimizer1 = optim.Adam(model1.parameters(), lr=lr)
optimizer2 = optim.Adam(model2.parameters(), lr=lr)
optimizer3 = optim.Adam(model3.parameters(), lr=lr)

# loss function
criterion = losses.SumTripletLoss(margin=1).to(device)

# trainer
trainer0 = trainers.BaseTrainer(model0, optimizer0, criterion, sampler, no_progressbar)
trainer1 = trainers.BaseTrainer(model1, optimizer1, criterion, sampler, no_progressbar)
trainer2 = trainers.BaseTrainer(model2, optimizer2, criterion, sampler, no_progressbar)
trainer3 = trainers.BaseTrainer(model3, optimizer3, criterion, sampler, no_progressbar)

# Base CML

In [6]:
trainer0.fit(n_batch, n_epoch, recall, valid_per_epoch)

In [7]:
# CML
trainer0.valid_scores

,Recall@5,Recall@10,Recall@50,epoch,losses
0,0.061124,0.125026,0.524287,0,NaN
0,0.209908,0.389403,0.815938,10,0.463673
0,0.344400,0.506366,0.855992,20,0.298063
0,0.381885,0.547917,0.876374,30,0.254923
0,0.396044,0.558414,0.882871,40,0.242846
0,0.400338,0.565996,0.887396,50,0.236934


# Gaussian

In [9]:
trainer2.fit(n_batch, n_epoch, recall, valid_per_epoch)

In [10]:
trainer2.valid_scores

,Recall@5,Recall@10,Recall@50,epoch,losses
0,0.060856,0.115124,0.515601,0,NaN
0,0.074680,0.149914,0.701594,10,0.465648
0,0.225349,0.362551,0.789108,20,0.304078
0,0.274630,0.426530,0.822549,30,0.258897
0,0.285588,0.444495,0.833560,40,0.243197
0,0.297883,0.452077,0.836460,50,0.236131


# Gamma

In [11]:
trainer3.fit(n_batch, n_epoch, recall, valid_per_epoch)

In [12]:
trainer3.valid_scores

,Recall@5,Recall@10,Recall@50,epoch,losses
0,0.064113,0.123277,0.525100,0,NaN
0,0.080098,0.189539,0.749208,10,0.465306
0,0.232996,0.392351,0.816009,20,0.303209
0,0.286423,0.451806,0.840743,30,0.256463
0,0.314927,0.475217,0.849844,40,0.243259
0,0.317811,0.479263,0.854333,50,0.235719
